<a href="https://colab.research.google.com/github/epalosh/NASA_SEES_2022/blob/main/Ground_Surface_Temperature_And_Elevation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Topics addressed:
  - reducers
  - loading and filtering image collections
  - loading, filtering, and editing vector data

## Setup

### Earth Engine setup

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=7N8ffAiyO5hX84gxzGjrcZDX2cyPCKcXLWR8SmkKYzw&tc=tzQy335ml-LCMmM_I1BxUd3xEHw9Jbq_Veys_8yhz3Y&cc=HmeoHW7UtLEzHZNFmHT337-SMu6nKm_mVy2AdyIwwUE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qiKXgBGwzWe3Q-4VQr8ja7DInbZNs8R_WnRgcOIWx0gyHdxXbU77rk

Successfully saved authorization token.


### geemap setup (for interactive map display)

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap.eefolium as geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap.eefolium as geemap
    print("geemap is now installed, imported and ready to use in Colab") 

#geemap.update_package()

geemap package not installed. Installing ...
geemap is now installed, imported and ready to use in Colab


In [ ]:
import folium


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

## Image stats by regions: A spatial reducer
###One way to calculated mean average temperature by county
In this code cell, we load a select set of counties from the TIGER file.<br>
Notice the trick used to select a subset of US counties in states that we choose to map.

In [ ]:
# load temp data through PRISM: 365 images per year
precipCollection = ee.ImageCollection('OREGONSTATE/PRISM/AN81d') \
                    .select('tmean') \
                    .filterDate('2021-01-01', '2021-12-31')

EcoRegions = ee.FeatureCollection('EPA/Ecoregions/2013/L3') 
   # .filter(ee.Filter.eq("STATEFP", "48"))


avgTemp = precipCollection.reduce(ee.Reducer.mean())


In [ ]:
SEES_CD_States = ['06','13','34','48','53'] # state FIPS codes that we want
# use the ee.Filter.inList('table property', list_variable) command to filter
# a new Feature Collection of only the states that we want to analyze
counties = ee.FeatureCollection("TIGER/2018/Counties") \
        .filter(ee.Filter.inList('STATEFP', SEES_CD_States))

# get mean temp values by county polygon
# Note the use of the reduceRegions and the mean Reducer.
# You can test other reduction methods like max, min, or median
# 4638.3 is the resolution of the PRISM dataset

countyTemp = avgTemp.reduceRegions(
    **{
        'collection': counties,
        'reducer': ee.Reducer.mean(),
        'scale': 4638.3,
       }
    )

ECO = ee.Image().paint(EcoRegions, 0, 1)

In [ ]:
tiger_co = ee.FeatureCollection("TIGER/2018/Counties")
print(type(tiger_co))

<class 'ee.featurecollection.FeatureCollection'>


Calculate the highest county mean temperature and show value.

In [ ]:
our_max_mean = countyTemp.aggregate_max('mean')
print('Max of mean county average temperature over a year = {} degrees celsius'.format(our_max_mean.getInfo()))

Max of mean county average temperature over a year = 24.157193714831102 degrees celsius


Calculate the lowest county mean temperature and show value.

In [ ]:
our_min_mean = countyTemp.aggregate_min('mean')
print('Min of mean county average temperature over a year = {} degrees celsius'.format(our_min_mean.getInfo()))

Min of mean county average temperature over a year = 6.547634910029122 degrees celsius


##Show the resulting mean average temperature by county

In [ ]:
palette = ['purple','blue','cyan','green','yellow','red']
# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte()
fills = empty.paint(
    **{
        'featureCollection': countyTemp,
        'color': 'mean',
       }
    )
Map = geemap.Map()
Map.setCenter(-99.67, 38.41, 5)
# Note using min and max from aggregate functions to set high and low visualization values.
Map.addLayer(avgTemp, {'min': 6.3, 'max': 24, 'palette': palette}, 'precip1')
Map.addLayer(fills, {'palette': palette, 'min': 2, 'max': 24}, 'colored fills')
Map.addLayer(avgTemp, {'palette': palette, 'min': 2, 'max': 24}, 'national colored fills')
Map.addLayer(ECO, {}, 'Counties In Texas') #Add ecoregion boundries
# you will need to code up the visualization of the counties.
# Map.addLayer(counties,{},'counties')
Map

NameError: ignored

##Vector to Raster Conversion
Why convert the county polygon vectors to a raster? <br>
**Vocabulary to remember:**

*   vector = point, line, polygon = *ee* Feature
*   raster = gridded data = *ee* Image
<br>
<br>
*ee* offers more visualization options for Images than for Features





In [ ]:
#Make an image out of the mean temperature attribute.
#print(type(countyPrecip))
meanTempImg = countyTemp.reduceToImage(properties=['mean'], reducer=ee.Reducer.first())

# Make a stylized palette color map
Map = geemap.Map()
Map.setCenter(-99.67, 38.41, 5)
Map.addLayer(avgTemp, {'min': 1, 'max': 24, 'palette': precipPal}, 'PRISM Temp')
Map.addLayer(meanTempImg, {'palette': palette, 'max': 24}, 'County means')

Map


In [ ]:
# Load SRTM Digital Elevation Model data.
image = ee.Image('CGIAR/SRTM90_V4')

# Define an SLD style of discrete intervals to apply to the image.
sld_intervals = """
    <RasterSymbolizer>
      <ColorMap type="intervals" extended="false" >
        <ColorMapEntry color="#0000ff" quantity="0" label="0"/>
        <ColorMapEntry color="#00ff00" quantity="1800" label="1-100" />
        <ColorMapEntry color="#007f30" quantity="3600" label="110-200" />
        <ColorMapEntry color="#30b855" quantity="5400" label="210-300" />
        <ColorMapEntry color="#ff0000" quantity="7200" label="310-400" />
        <ColorMapEntry color="#ffff00" quantity="9000" label="410-1000" />
      </ColorMap>
    </RasterSymbolizer>"""

# Define an sld style color ramp to apply to the image.
sld_ramp = """
    <RasterSymbolizer>
      <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#0000ff" quantity="0" label="0"/>
        <ColorMapEntry color="#00ff00" quantity="1800" label="100" />
        <ColorMapEntry color="#007f30" quantity="3600" label="200" />
        <ColorMapEntry color="#30b855" quantity="5400" label="300" />
        <ColorMapEntry color="#ff0000" quantity="7200" label="400" />
        <ColorMapEntry color="#ffff00" quantity="9000" label="500" />
      </ColorMap>
    </RasterSymbolizer>"""

# Define a map centered on the United States.
map_sld_interval = folium.Map(location=[40.413, -99.229], zoom_start=5)

# Add the image layers to the map and display it.
map_sld_interval.add_ee_layer(
    image.sldStyle(sld_intervals), None, 'SLD intervals')
map_sld_interval.add_ee_layer(image.sldStyle(sld_ramp), None, 'SLD ramp')
display(map_sld_interval.add_child(folium.LayerControl()))